<a href="https://colab.research.google.com/github/christian-thomas-schmidt/Christian_Portfolio/blob/main/Assignment_1_Helper_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --ignore-installed -q pyspark==3.2.1

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 52.5 MB/s 


In [3]:
import os
import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)



/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:

# requesting Spark context
# sc = SparkContext(appName="HW1_Task1")

In [4]:
#Exception Handling and removing wrong datalines
def isfloat(value):
    try:
        float(value)
        return True
 
    except:
         return False

In [5]:
#For example, remove lines if they don’t have 16 values and 
# checking if the trip distance and fare amount is a float number
# checking if the trip duration is more than a minute, trip distance is more than 0.1 miles, 
# fare amount and total amount are more than 0.1 dollars
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[4])> 60 and float(p[5])>0.10 and float(p[11])> 0.10 and float(p[16])> 0.10):
                return p



In [6]:
# Upload the data
# At this point you have to upload your data to google colab
! wget -q https://storage.googleapis.com/cs777_spring_2022_public/taxi-data-sorted-verysmall.csv

In [7]:
pwd

'/content'

# New Section

In [8]:
ls -la

total 1896
drwxr-xr-x 1 root root    4096 May 20 02:20 ./
drwxr-xr-x 1 root root    4096 May 20 02:14 ../
drwxr-xr-x 4 root root    4096 May 17 13:38 .config/
drwxr-xr-x 1 root root    4096 May 17 13:39 sample_data/
-rw-r--r-- 1 root root 1921037 Mar 11 15:07 taxi-data-sorted-verysmall.csv


In [9]:
# Set your file path here 
path="file:///content/"

# You have 1 files 

testFile= path + "taxi-data-sorted-verysmall.csv"


testDataFrame = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep =",").load(testFile)

testDataFrame.show()

+--------------------------------+--------------------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+---+----+------+------+------+------+----+
|07290D3599E7A0D62097A346EFCC1FB5|E7750A37CAB07D0DFF0AF7E3573AC141|2013-01-01 00:00:00|2013-01-01 00:02:00|120|0.44|-73.956528|40.716976|-73.962440|40.715008|CSH|3.50|0.5012|0.5013|0.0014|0.0015|4.50|
+--------------------------------+--------------------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+---+----+------+------+------+------+----+
|            22D70BF00EEB0ADC8...|            3FF2709163DE7036F...|2013-01-01 00:02:00|2013-01-01 00:02:00|  0| 0.0|       0.0|      0.0|       0.0|      0.0|CSH|27.0|   0.0|   0.5|   0.0|   0.0|27.5|
|            0EC22AAF491A8BD91...|            778C92B26AE78A9EB...|2013-01-01 00:01:00|2013-01-01 00:03:00|120|0.71|-73.973145|40.752827|-73.965897|40.760445|CSH| 4.0|   0.5|   0.5|   0.0|   0.0| 

In [10]:
testRDD = testDataFrame.rdd.map(tuple)
testRDD.take(1)

[('22D70BF00EEB0ADC83BA8177BB861991',
  '3FF2709163DE7036FCAA4E5A3324E4BF',
  '2013-01-01 00:02:00',
  '2013-01-01 00:02:00',
  0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  'CSH',
  27.0,
  0.0,
  0.5,
  0.0,
  0.0,
  27.5)]

In [37]:

# calling isfloat and correctRows functions to cleaning up data
taxilinesCorrected = testRDD.filter(correctRows)
taxilinesCorrected.take(1)

[('0EC22AAF491A8BD91F279350C2B010FD',
  '778C92B26AE78A9EBDF96B49C67E4007',
  '2013-01-01 00:01:00',
  '2013-01-01 00:03:00',
  120,
  0.71,
  -73.973145,
  40.752827,
  -73.965897,
  40.760445,
  'CSH',
  4.0,
  0.5,
  0.5,
  0.0,
  0.0,
  5.0)]

In [69]:
counts = taxilinesCorrected.map(lambda x: (x[0], 1)).reduceByKey(add)
counts.take(10)

[('0EC22AAF491A8BD91F279350C2B010FD', 1),
 ('1390FB380189DF6BBFDA4DC847CAD14F', 3),
 ('3B4129883A1D05BE89F2C929DE136281', 2),
 ('468244D1361B8A3EB8D206CC394BC9E9', 2),
 ('5F78CC6D4ECD0541B765FECE17075B6F', 4),
 ('6BA29E9A69B10F218C1509BEDD7410C2', 4),
 ('C306CAC565429C12852164EB38175736', 4),
 ('DD467ED2E7DDB5C8B9B918469604E54F', 2),
 ('120E700FE35B2DDBEA4D64CCCF02C808', 3),
 ('256C6A3CA47F81497C1F5038438B54B2', 2)]

In [70]:
counts.top(10, lambda x: x[1])

[('696321779D687411F2E5DF6991E9D474', 7),
 ('9B5BE818E3C04896687FFD3C4BF5555F', 6),
 ('67424A3EE2C533FC4522FFB2C0903E9F', 6),
 ('06808F2B069EBFA72B16977B7C4672C4', 6),
 ('2F916CFD5E51E9A620F3D4F9D65D2A7B', 6),
 ('74AB002C9DEB2BA209464F2960E630B9', 6),
 ('A15D70A795948BBF608D89738C4EFCF6', 6),
 ('7CC2C787A937EE1F874D2FFD6BBCD2F0', 5),
 ('1F9E2C27335E24C6A72069A4A550E3D5', 5),
 ('8669D9002FF4D63D433C5D9CCA710D7C', 5)]

In [65]:
taxilinesCorrected.take(1)

[('0EC22AAF491A8BD91F279350C2B010FD',
  '778C92B26AE78A9EBDF96B49C67E4007',
  '2013-01-01 00:01:00',
  '2013-01-01 00:03:00',
  120,
  0.71,
  -73.973145,
  40.752827,
  -73.965897,
  40.760445,
  'CSH',
  4.0,
  0.5,
  0.5,
  0.0,
  0.0,
  5.0)]

In [115]:
time_driver = taxilinesCorrected.map(lambda x: (x[0], x[4]/60)).reduceByKey(lambda x, y : x+y)
money_driver = taxilinesCorrected.map(lambda x: (x[0], x[16])).reduceByKey(lambda x, y : x+y)


In [121]:
test = taxilinesCorrected.map(lambda x: (x[0], x[16]/x[4]*60)).reduceByKey(lambda x, y : x+y)


[('0EC22AAF491A8BD91F279350C2B010FD', 2.5),
 ('1390FB380189DF6BBFDA4DC847CAD14F', 4.824999999999999),
 ('3B4129883A1D05BE89F2C929DE136281', 3.611111111111111),
 ('468244D1361B8A3EB8D206CC394BC9E9', 2.5719696969696972),
 ('5F78CC6D4ECD0541B765FECE17075B6F', 7.489285714285715)]

In [122]:
test.top(10, lambda x: x[1])

[('696321779D687411F2E5DF6991E9D474', 11.591666666666667),
 ('54E0A7E84EAFDF6D0C70DC8E8A272FD9', 11.533333333333331),
 ('8FFD1464685AAE39ECD77415EE4E11A1', 10.57840909090909),
 ('A15D70A795948BBF608D89738C4EFCF6', 10.566666666666666),
 ('9B5BE818E3C04896687FFD3C4BF5555F', 10.129852813852814),
 ('93191F5C828803BF328DFE590A90C20B', 10.027777777777779),
 ('17A181BBBA33F2D7D45D975CD9DAF2E2', 9.66904761904762),
 ('DB18E9E94EB754CF15E66B03852EA17B', 9.566666666666666),
 ('F2751CE89ED7E006D8E9C243FC31D01C', 9.366666666666667),
 ('CEABFF0193174D655262201BABBADEF0', 9.35)]

[('0EC22AAF491A8BD91F279350C2B010FD', 5.0),
 ('1390FB380189DF6BBFDA4DC847CAD14F', 5.0),
 ('3B4129883A1D05BE89F2C929DE136281', 5.0),
 ('468244D1361B8A3EB8D206CC394BC9E9', 7.5),
 ('5F78CC6D4ECD0541B765FECE17075B6F', 8.3)]